In [1]:
from napari import Viewer
from tifffile import imread
import numpy as np

from skimage import io, filters, measure, morphology, segmentation

import pandas as pd

from matplotlib import pyplot as plt

In [2]:
file = '../raw_data/BBBC001_v1_images_tif/human_ht29_colon_cancer_1_images/AS_09125_050118150001_A03f00d0.tif'

In [3]:
image = imread(file)

In [4]:
print(image.shape, image.dtype)

(512, 512) uint8


In [5]:
viewer = Viewer()

In [6]:
viewer.add_image(image)

<Image layer 'image' at 0x7f36782824b0>

In [7]:
# Step 1: Apply Otsu's thresholding
threshold_value = filters.threshold_otsu(image)
binary_image = image > threshold_value

In [8]:
# Step 2: Remove small objects/noise if necessary
binary_image_cleaned = morphology.remove_small_objects(binary_image, min_size=64)
# Step 3: Fill holes within the binary mask
binary_image_filled = morphology.remove_small_holes(binary_image_cleaned, area_threshold=64)

In [9]:
# Step 3: Label the connected components (nuclei)
labeled_image = segmentation.clear_border(measure.label(binary_image_filled))

In [10]:
viewer.add_labels(labeled_image)

<Labels layer 'labeled_image' at 0x7f36546978f0>

In [11]:
properties = measure.regionprops_table(labeled_image, intensity_image=image, 
                                       properties=['label', 'area', 'mean_intensity'])

# Convert to pandas DataFrame for easy export
properties_df = pd.DataFrame(properties)

In [12]:
properties_df

,label,area,mean_intensity
0,3,82.0,42.621951
1,4,77.0,79.948052
2,5,135.0,67.733333
3,6,91.0,42.000000
4,7,164.0,56.518293
...,...,...,...
273,282,85.0,52.435294
274,283,93.0,60.279570
275,284,80.0,58.600000
276,285,153.0,75.228758


In [13]:
properties_df.to_csv('measurements.csv', index=False)